<a href="https://colab.research.google.com/github/Ayanlola2002/DATA-SCIENCE-PROJECTS/blob/master/NeedEnergyAPI_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [242]:
#mounting Gdrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [243]:
import pandas as pd
import numpy as np
from datetime import date, datetime
import holidays

In [244]:
df_5min=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_five_min_id_47803.csv')
df_hourly=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_hourly_id_47803.csv')
df_daily=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_daily_id_47803.csv')
df_monthly=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_monthly_id_47803.csv')
df_public_holiday=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/public_holidays_weekends_ZIM.csv')


In [245]:
##Wrangling Helper Functions to Use
"""
This function do the reading of data file from file path(i.e drive)
"""
def do_read_data(datafile, dateCol):
    ''' Read the excel file'''
    df = pd.read_excel(datafile, index_col= dateCol, parse_dates=True)
    return df

"""
This function Shows Information of the Data file
Like The Header columns, Dimension of Data
"""

def do_data_information(datafile, info = False, head = True, shape = True):
    ''' Look at the info of the data'''
    
    if not isinstance(datafile, pd.DataFrame):
        df = read_data(datafile)
    else:
        df = datafile
    
    ## Information of data
    if info:
        df.info()
        
    print('--------------------------------------------------------------------')
    print('First rows')
    # Show first rows
    if head:
        print(df.head())
        
    print('--------------------------------------------------------------------')
    print('Dimension of the data')
    ## Dimension of data
    if shape:
        print(f'This data has {df.shape[0]} rows and {df.shape[1]} variables') 
        
###This function returns the number of distinct values from any columns       
def do_unique_values(df, col):
    ''' Find the unique values of a columns'''
    return df[col].nunique()


###This function statistically describe the data values
def do_description(df, col = None):
    ''' Give the statistic summary'''
    
    ## Summary statistic
    if col is not None:
        stat = df[col].describe()
    else:
        stat = df.describe()

    ## Change the name of the 50% index to median
    idx = stat.index.tolist()
    idx[5] = 'median'
    stat.index = idx
    return stat

###This function finds % Missing values
def check_missing_value(data):
    ## percentage of missing values
    n = data.isnull().sum().sort_values(ascending=False)/ len(data) * 100
    return n

def check_missing_values_table(df):
  mis_val = df.isnull().sum()
  mis_val_percent = 100 * df.isnull().sum() / len(df)
  mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
  mis_val_table_ren_columns = mis_val_table.rename(
  columns = {0 : 'Missing Values', 1 : '% of Total Values'})
  mis_val_table_ren_columns = mis_val_table_ren_columns[
      mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
  '% of Total Values', ascending=False).round(1)
  print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
      "There are " + str(mis_val_table_ren_columns.shape[0]) +
        " columns that have missing values.")
  return mis_val_table_ren_columns


###This function finds the amount of each values in a dataset
def do_valueCount(data, col, perc = False):
    ''' calculate the proportion in fraction or percentage'''
    if not perc:
      return data[col].value_counts()
    else:
      return data[col].value_counts(normalize = True).mul(100).round(1).astype(str) + '%'    
### This function count number of unique values for all columns in a dataset
def do_nunique_value(df):
  print('The Numver of unique values per columns are as follows')
  print(df.agg(['nunique']).T)


###For date time conversion start here   
def do_date_col_convertion(df, column_name):
  df[column_name]=pd.to_datetime(df[column_name])
  return df[column_name]

def do_time_col_convertion(df, column_name):
    return pd.to_timedelta(df[column_name])


def do_datepart_extraction(data):
  data['date'] = pd.to_datetime(data['date'])
  data['dateOnly']=data['date'].dt.date
  data['year'] = data['date'].dt.year
  data['quarter']=data['date'].dt.quarter
  data['month'] = data['date'].dt.month
  data['weekday'] = data['date'].dt.weekday
  data['day_name'] = data['date'].dt.day_name()
  data['dayofyear'] = data['date'].dt.dayofyear
  data['day'] =data['date'].dt.day
  data['hour'] = data['date'].dt.hour
  data['minute'] =data['date'].dt.minute
  #data['Season']=data['month'].apply(do_year_season)
  data['season'] = data.date.map(do_season_of_date)
  data['day_section']=data['hour'].apply(do_day_sections)
  return data
   

def do_season_of_date(date):
  year = str(date.year)
  seasons = {'spring': pd.date_range(start='21/03/'+year, end='20/06/'+year),
              'summer': pd.date_range(start='21/06/'+year, end='22/09/'+year),
              'autumn': pd.date_range(start='23/09/'+year, end='20/12/'+year)}
  if date in seasons['spring']:
    return 'spring'
  if date in seasons['summer']:
    return 'summer'
  if date in seasons['autumn']:
    return 'autumn'
  else:
    return 'winter'

# Assuming df has a date column of type `datetime`
#df['season'] = df.date.map(season_of_date)


def do_day_sections(x):
  if (x > 4) and (x <= 8):
    return 'Early Morning'
  elif (x > 8) and (x <= 12 ):
    return 'Morning'
  elif (x > 12) and (x <= 16):
    return'Noon'
  elif (x > 16) and (x <= 20) :
    return 'Evening'
  elif (x > 20) and (x <= 24):
    return'Night'
  elif (x <= 4):
    return'Late Night'




def get_holidays(startYear = 2018, endYear = 2025, countryCode = 'ZA'):
  """
    Takes in a start and end date, and start and end year.
    
    Produces a dataframe with a daily date and columns:
    holiday - 'Y' for holiday
    holidayName - name of the holiday if holiday is 'Y'
    
    Returns a dataframe
  """
  holidayDict = {}
  for i in range(startYear, endYear):
    for date, name in sorted(holidays.CountryHoliday(countryCode,years=[i]).items()):
      holidayDict[date] = name
      holiday_df = pd.DataFrame(list(holidayDict.items()),columns = ['day','holidayName'])
      holiday_df['day'] = pd.to_datetime(holiday_df['day'], format='%Y-%M-%d').dt.date
    return holiday_df
    

    
def get_days(start = '1/1/2018',startYear = 2018, end = '31/12/2025',endYear = 2025, countryCode = 'ZA'):
    """
    Takes in a start and end date, and start and end year.
    
    Produces a dataframe with a daily date and columns:
    weekend - 'Y' for weendend and 'N' for workday
    dayOfweek - numerical day of the week identifier 0 for monday
    weekNum - numerical number of the week 
    holiday - 'Y' for holiday
    holidayName - name of the holiday if holiday is 'Y'
    
    Returns a dataframe
    """
    
    #generate the range of daily dates
    dates = pd.date_range(start = start, end = end)
    date_df = pd.DataFrame(dates, columns = ['day'])
    date_df['day'] = pd.to_datetime(date_df['day'], format='%Y-%M-%d')
    country_holidays = get_holidays(startYear = startYear, endYear = endYear, countryCode = countryCode)
    
    date_df['dayName'] = pd.DatetimeIndex(date_df['day']).day_name()
    date_df['dayOfWeek'] = date_df['day'].dt.dayofweek
    date_df['weekend'] = date_df['dayOfWeek'].apply(lambda x: 'Y' if x>4 else 'N')
    date_df['weekNum'] = date_df['day'].dt.week
    date_df['holiday'] = date_df['day'].apply(lambda x: 'Y' if x in country_holidays['day'].values else 'N')
    date_df['day'] = date_df['day'].dt.date
    date_df = date_df.merge(country_holidays, on='day', how='left', indicator=False)
    
    date_df.to_csv(f'../data/public_holidays_weekends.csv', index=False)
    return date_df

###Functions to impute nan values for numerical varriable
def Impute_nan(df):
  numericalcol = [f for f in df_5min.columns if df_5min.dtypes[f] != 'object']
  missing_col=df_5min[numericalcol].columns[df_5min[numericalcol].isnull().any()].tolist()
  #Technique 1: Using mean to impute the missing values
  for i in missing_col:
    df.loc[df.loc[:,i].isnull(),i]=df.loc[:,i].mean()

def Impute_na(df):
  numericalcol = [f for f in df_5min.columns if df_5min.dtypes[f] != 'object']
  missing_col=df_5min[numericalcol].columns[df_5min[numericalcol].isnull().any()].tolist()
  #Technique 1: Using mean to impute the missing values
  for i in missing_col:
     df[i].fillna(mean,inplace=True)




#df['holidays']=df['date'].apply(lambda(x: 'Hols' if x in get_holidays()['day'].values else 'Non-Hols'))



In [246]:

#Data Description and Information for df_5min
print(do_data_information(df_5min))
print(do_data_information(df_hourly))
print(do_data_information(df_daily))
print(do_data_information(df_monthly))
print(do_data_information(df_public_holiday))

--------------------------------------------------------------------
First rows
                  date  ...                                   voltageHarmonics
0  2020-03-15 18:15:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
1  2020-03-15 18:20:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
2  2020-03-15 18:25:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
3  2020-03-15 18:30:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
4  2020-03-15 18:35:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...

[5 rows x 15 columns]
--------------------------------------------------------------------
Dimension of the data
This data has 95649 rows and 15 variables
None
--------------------------------------------------------------------
First rows
                  date  ...                                   voltageHarmonics
0  2020-03-15 18:00:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
1  2020-03-15 19:00:00  ...  [[], [], [], [], [

Data Quality Information:
df_5min : The data has 95649 rows and 15 features

In [247]:
#Checking Missing Values by percentage
print(check_missing_values_table(df_5min))
print(check_missing_values_table(df_daily))
print(check_missing_values_table(df_hourly))
print(check_missing_values_table(df_monthly))
print(check_missing_values_table(df_public_holiday))


Your selected dataframe has 15 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []
Your selected dataframe has 15 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []
Your selected dataframe has 15 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []
Your selected dataframe has 15 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []
Your selected dataframe has 7 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []


In [248]:
#Checking number of Unique values
print(do_nunique_value(df_5min))
print(do_nunique_value(df_hourly))
print(do_nunique_value(df_daily))
print(do_nunique_value(df_monthly))
print(do_nunique_value(df_public_holiday))

The Numver of unique values per columns are as follows
                  nunique
date                95649
timestamp           95649
consumption         76377
solar               50268
alwaysOn              955
gridImport          66188
gridExport              1
selfConsumption         2
selfSufficiency      5594
active              95512
reactive            95595
voltages              389
phaseVoltages         389
currentHarmonics        1
voltageHarmonics        1
None
The Numver of unique values per columns are as follows
                  nunique
date                 7979
timestamp            7979
consumption          7963
solar                7876
alwaysOn              714
gridImport           7946
gridExport              1
selfConsumption         2
selfSufficiency      3092
active               7979
reactive             7979
voltages              280
phaseVoltages         280
currentHarmonics        1
voltageHarmonics        1
None
The Numver of unique values per columns are as f

In [249]:
#Feature Engineering
df_5min=do_datepart_extraction(df_5min)


In [250]:
df_5min.head()

,date,timestamp,consumption,solar,alwaysOn,gridImport,gridExport,selfConsumption,selfSufficiency,active,reactive,voltages,phaseVoltages,currentHarmonics,voltageHarmonics,dateOnly,year,quarter,month,weekday,day_name,dayofyear,day,hour,minute,season,day_section
0,2020-03-15 18:15:00,1584288900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.7, 0.0, -81.6, -60.0, 30.2, -230.6, -0...","[0.5, 0.9, 0.0, 55.4, 132.4, 14.0, 137.5, 0.2,...","[244.5, None, None]","[244.5, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-15,2020,1,3,6,Sunday,75,15,18,15,winter,Evening
1,2020-03-15 18:20:00,1584289200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.8, 0.0, -82.2, -58.5, 30.5, -233.1, -0...","[0.5, 0.9, 0.0, 55.7, 131.9, 14.3, 144.3, 0.2,...","[246.1, None, None]","[246.1, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-15,2020,1,3,6,Sunday,75,15,18,20,winter,Evening
2,2020-03-15 18:25:00,1584289500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 28.5, -26.5, -74.4, -79.2, 55.5, -228.5,...","[0.5, 19.0, 8.8, 79.6, 137.6, 29.4, 175.0, 0.4...","[246.4, None, None]","[246.4, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-15,2020,1,3,6,Sunday,75,15,18,25,winter,Evening
3,2020-03-15 18:30:00,1584289800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.5, 21.3, -18.6, -76.9, -54.7, 48.7, -227.8,...","[2.2, 13.3, 6.5, 72.4, 135.9, 24.8, 152.8, 0.8...","[246.0, None, None]","[246.0, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-15,2020,1,3,6,Sunday,75,15,18,30,winter,Evening
4,2020-03-15 18:35:00,1584290100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.7, 0.0, -82.6, -33.2, 30.7, 96.6, 0.2,...","[0.5, 0.9, 0.0, 55.2, 128.1, 13.9, 87.7, 0.7, ...","[246.2, None, None]","[246.2, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-15,2020,1,3,6,Sunday,75,15,18,35,winter,Evening


In [251]:
df_5min.date.min(),df_5min.date.max()

(Timestamp('2020-03-15 18:15:00'), Timestamp('2021-02-12 01:55:00'))

In [252]:
df_5min.consumption.min(),df_5min.consumption.max()

(0.0, 939.27)

In [253]:
#checking for days where there weere no consumptions
df_5min[df_5min.consumption==0].value_counts().count(),df_5min[df_5min.consumption==0].value_counts()

(11,
 date                 timestamp      consumption  solar  alwaysOn  gridImport  gridExport  selfConsumption  selfSufficiency  active                                                                                                                                                                       reactive                                                                                                                                                              voltages             phaseVoltages        currentHarmonics                                                                                                  voltageHarmonics                                                                                                  dateOnly    year  quarter  month  weekday  day_name  dayofyear  day  hour  minute  season  day_section
 2020-03-15 19:05:00  1584291900000  0.0          0.0    0.0       0.0         0.0         0.0              0.0              [0.1, 20.2, 0.0, 35.8, 0.0, 46.9,

In [254]:
#checking for days where there weere no consumptions
df_5min[df_5min.consumption>0].value_counts().count(),df_5min[df_5min.consumption>0].value_counts()

(95638,
 date                 timestamp      consumption  solar   alwaysOn  gridImport  gridExport  selfConsumption  selfSufficiency  active                                                                                                                                                               reactive                                                                                                                                                                voltages             phaseVoltages        currentHarmonics                                                                                                  voltageHarmonics                                                                                                  dateOnly    year  quarter  month  weekday  day_name  dayofyear  day  hour  minute  season  day_section  
 2021-02-12 01:55:00  1613087700000  153.934      46.152  1006.212  107.781     0.0         100.0            29.98            [0.1, 3.6, -0.1, None, None, Non

In [255]:
'''data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['quarter']=data['date'].dt.quarter
data['month'] = data['date'].dt.month
data['weekday'] = data['date'].dt.weekday
data['day_name'] = data['date'].dt.day_name()
data['dayofyear'] = data['date'].dt.dayofyear
data['day'] =data['date'].dt.day
data['hour'] = data['date'].dt.hour
data['minute'] =data['date'].dt.minute
#data['Season']=data['month'].apply(do_year_season)
data['season'] = data.date.map(do_season_of_date)
data['day_section']=data['hour'].apply(do_day_sections)'''

"data['date'] = pd.to_datetime(data['date'])\ndata['year'] = data['date'].dt.year\ndata['quarter']=data['date'].dt.quarter\ndata['month'] = data['date'].dt.month\ndata['weekday'] = data['date'].dt.weekday\ndata['day_name'] = data['date'].dt.day_name()\ndata['dayofyear'] = data['date'].dt.dayofyear\ndata['day'] =data['date'].dt.day\ndata['hour'] = data['date'].dt.hour\ndata['minute'] =data['date'].dt.minute\n#data['Season']=data['month'].apply(do_year_season)\ndata['season'] = data.date.map(do_season_of_date)\ndata['day_section']=data['hour'].apply(do_day_sections)"

In [256]:
df_public_holiday.head()

,day,dayName,dayOfWeek,weekend,weekNumber,holiday,holidayName
0,2018/01/01,Monday,0,N,1,Y,New Year's Day
1,2018/01/02,Tuesday,1,N,1,N,
2,2018/01/03,Wednesday,2,N,1,N,
3,2018/01/04,Thursday,3,N,1,N,
4,2018/01/05,Friday,4,N,1,N,


In [257]:
df_public_holiday.holidayName.value_counts()

                                         2460
Easter Sunday                               7
Good Friday                                 7
Robert Mugabe National Youth Day            7
Holy Saturday                               7
Africa Day                                  7
Zimbabwe Defence Forces Day                 7
Heroes' Day                                 7
Christmas Day                               6
New Year's Day                              6
Easter Monday                               6
Worker's Day                                6
National Unity Day                          5
Independence Day                            5
Boxing Day                                  5
National Unity Day (observed)               2
Independence Day (observed)                 1
Solidarity Day Against Sanctions            1
Christmas Day (observed) / Boxing Day       1
Worker's Day (observed)                     1
New Year's Day (observed)                   1
Zimbabwean General Election       

In [258]:
df_public_holiday.isnull().sum()

day            0
dayName        0
dayOfWeek      0
weekend        0
weekNumber     0
holiday        0
holidayName    0
dtype: int64

In [259]:
df_public_holiday['day'] = pd.to_datetime(df_public_holiday['day'],format='%Y-%m-%d')

In [260]:
mapping = dict(df_public_holiday[['day','holidayName']].values)



In [261]:
mapping

{Timestamp('2018-01-01 00:00:00'): "New Year's Day",
 Timestamp('2018-01-02 00:00:00'): ' ',
 Timestamp('2018-01-03 00:00:00'): ' ',
 Timestamp('2018-01-04 00:00:00'): ' ',
 Timestamp('2018-01-05 00:00:00'): ' ',
 Timestamp('2018-01-06 00:00:00'): ' ',
 Timestamp('2018-01-07 00:00:00'): ' ',
 Timestamp('2018-01-08 00:00:00'): ' ',
 Timestamp('2018-01-09 00:00:00'): ' ',
 Timestamp('2018-01-10 00:00:00'): ' ',
 Timestamp('2018-01-11 00:00:00'): ' ',
 Timestamp('2018-01-12 00:00:00'): ' ',
 Timestamp('2018-01-13 00:00:00'): ' ',
 Timestamp('2018-01-14 00:00:00'): ' ',
 Timestamp('2018-01-15 00:00:00'): ' ',
 Timestamp('2018-01-16 00:00:00'): ' ',
 Timestamp('2018-01-17 00:00:00'): ' ',
 Timestamp('2018-01-18 00:00:00'): ' ',
 Timestamp('2018-01-19 00:00:00'): ' ',
 Timestamp('2018-01-20 00:00:00'): ' ',
 Timestamp('2018-01-21 00:00:00'): ' ',
 Timestamp('2018-01-22 00:00:00'): ' ',
 Timestamp('2018-01-23 00:00:00'): ' ',
 Timestamp('2018-01-24 00:00:00'): ' ',
 Timestamp('2018-01-25 00:0

In [262]:
df_5min['Public_Holiday'] = df_5min['dateOnly'].map(mapping)

In [263]:
df_5min['Public_Holiday']=df_5min['Public_Holiday'].apply(lambda x:np.NaN if isinstance(x, str) and (x.isspace() or not x) else x).replace(np.NaN,'Non_Holiday')
#df_5min=df_5min.apply(df_5min['Public_Holiday'].astype(str).apply(lambda x: x.strip()).replace('', np.nan))

In [264]:
df_5min.sample(5)

,date,timestamp,consumption,solar,alwaysOn,gridImport,gridExport,selfConsumption,selfSufficiency,active,reactive,voltages,phaseVoltages,currentHarmonics,voltageHarmonics,dateOnly,year,quarter,month,weekday,day_name,dayofyear,day,hour,minute,season,day_section,Public_Holiday
59382,2020-10-09 00:50:00,1602197400000,295.302,57.418,1825.500,237.884,0.0,100.0,19.44,"[0.1, 3.5, 0.0, None, None, None, None, 0.3, N...","[0.4, 0.8, 0.0, None, None, None, None, 0.2, N...","[239.1, None, None]","[239.1, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-10-09,2020,4,10,4,Friday,283,9,0,50,winter,Late Night,Non_Holiday
78051,2020-12-12 21:20:00,1607800800000,297.315,48.303,858.132,249.012,0.0,100.0,16.25,"[0.1, 3.7, -0.1, None, None, None, None, 0.3, ...","[-0.5, 0.9, 0.0, None, None, None, None, -0.2,...","[243.4, None, None]","[243.4, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-12-12,2020,4,12,5,Saturday,347,12,21,20,winter,Night,Non_Holiday
6552,2020-04-07 13:30:00,1586259000000,302.440,58.088,1565.724,244.351,0.0,100.0,19.21,"[0.1, 3.6, 0.0, None, None, None, None, 0.3, N...","[0.4, 0.8, 0.0, None, None, None, None, 0.2, N...","[240.3, None, None]","[240.3, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-04-07,2020,2,4,1,Tuesday,98,7,13,30,winter,Noon,Non_Holiday
19350,2020-05-22 01:15:00,1590102900000,138.319,48.566,1576.812,89.752,0.0,100.0,35.11,"[0.1, 3.7, 0.0, None, None, None, None, 0.4, N...","[0.5, 0.9, 0.0, None, None, None, None, 0.2, N...","[243.2, None, None]","[243.2, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-05-22,2020,2,5,4,Friday,143,22,1,15,winter,Late Night,Non_Holiday
56415,2020-09-28 17:35:00,1601307300000,299.266,80.450,2716.248,218.816,0.0,100.0,26.88,"[0.1, 3.4, 0.0, None, None, None, None, 0.3, N...","[0.4, 0.8, 0.0, None, None, None, None, 0.2, N...","[236.7, None, None]","[236.7, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-09-28,2020,3,9,0,Monday,272,28,17,35,winter,Evening,Non_Holiday


In [265]:
df_5min['Public_Holiday'].unique()

array(['Non_Holiday', 'Good Friday', 'Holy Saturday', 'Easter Sunday',
       'Easter Monday', 'Independence Day', "Worker's Day", 'Africa Day',
       "Heroes' Day", 'Zimbabwe Defence Forces Day', 'National Unity Day',
       'Christmas Day', 'Boxing Day', "New Year's Day"], dtype=object)

In [266]:
df_5min.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95649 entries, 0 to 95648
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              95649 non-null  datetime64[ns]
 1   timestamp         95649 non-null  int64         
 2   consumption       95649 non-null  float64       
 3   solar             95649 non-null  float64       
 4   alwaysOn          95649 non-null  float64       
 5   gridImport        95649 non-null  float64       
 6   gridExport        95649 non-null  float64       
 7   selfConsumption   95649 non-null  float64       
 8   selfSufficiency   95649 non-null  float64       
 9   active            95649 non-null  object        
 10  reactive          95649 non-null  object        
 11  voltages          95649 non-null  object        
 12  phaseVoltages     95649 non-null  object        
 13  currentHarmonics  95649 non-null  object        
 14  voltageHarmonics  9564

#DATA QUALITY CHECK:
*Data summary:
* Raw_NeedEnergyApi_5min :
Raw_Records: 95649 with 15 columns
Cleaned(Current) :95649,

* Raw_NeedEnergyApi_hourly :
Raw_Record :7979 with 15 columns
Cleaned(Current) :7979,
* Raw_NeedEnergyApi_Daily :
Raw_Records: 335 with 15 columns
Cleaned (current):335
* Raw_NeedEnergyApi_Monthly :
Raw_records:12 with 15 columns
cleaned =12
* BASED ON THE FOLLOWING QUALITY DIMENSIONS:(ACCURACY,COMPELETENESS,VALUES FREE FROM CONTRADICTION)
THE NEEDENERGY API 5MIN,HOURLY,DAILTY,MONTHLY DATASET has zero missing values.More data required for predictive analysis,

## Feature Engineering:
More features were extracted and created from the date column feature:
* New features Generated :day,hour,month,year,day_name,
seasons(winter,autumn,spring),day sections(Night,Evening,Afternoon)

* Others:Jupyter starter files for 
Generalized Helper functions were created to generate seasons,day_sections,day,month,year,hour,quarter,were created for use for anyone willing to start-up cleaning or other task activities

* In-Progress Helper function
Public-holiday and many more for solar-iiradiance dataset.
